### Installation
Install the packages required for executing this notebook.

In [1]:
import os

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME") and not os.getenv("VIRTUAL_ENV")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"

! pip3 install --upgrade google-cloud-aiplatform {USER_FLAG} -q
! pip3 install -U google-cloud-storage {USER_FLAG} -q
! pip3 install {USER_FLAG} kfp google-cloud-pipeline-components --upgrade -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kfp 1.8.14 requires google-cloud-storage<2,>=1.20.0, but you have google-cloud-storage 2.5.0 which is incompatible.
google-cloud-pipeline-components 1.0.25 requires google-cloud-storage<2,>=1.20.0, but you have google-cloud-storage 2.5.0 which is incompatible.


## Restart the kernel
Once you've installed the additional packages, you need to restart the notebook kernel so it can find the packages.

In [2]:
# import os

# if not os.getenv("IS_TESTING"):
#     # Automatically restart kernel after installs
#     import IPython

#     app = IPython.Application.instance()
#     app.kernel.do_shutdown(True)

Check the versions of the packages you installed. The KFP SDK version should be >=1.6.

In [3]:
! python3 -c "import kfp; print('KFP SDK version: {}'.format(kfp.__version__))"

KFP SDK version: 1.8.14


In [4]:
import kfp
import typing
from typing import Dict
from typing import NamedTuple
from kfp.v2 import dsl
from kfp.v2.dsl import (Artifact,
                        Dataset,
                        Input,
                        Model,
                        Output,
                        Metrics,
                        ClassificationMetrics,
                        component, 
                        OutputPath, 
                        InputPath)
import google.cloud.aiplatform as aip
from google_cloud_pipeline_components import aiplatform as gcc_aip
from kfp.v2.components import importer_node
from google_cloud_pipeline_components.types import artifact_types

#### Project and Pipeline Configurations

In [5]:
#The Google Cloud project that this pipeline runs in.
project_id = "de2022-362707"
# The region that this pipeline runs in
region = "us-central1"
# Specify a Cloud Storage URI that your pipelines service account can access. The artifacts of your pipeline runs are stored within the pipeline root.
pipeline_root_path = "gs://de_jads_temp_2093373"

#### Create Pipeline Components

We can create a component from Python functions (inline) and from a container. We will first try inline python functions. 

Step 1: Define the python function

Step 2:  Use **kfp.components.create_component_from_func** build the component. This function takes four parameters.

**1.func**: The Python function to convert.

**2.base_image**: (Optional.) Specify the Docker container image to run this function in. 

**3.output_component_file**: (Optional.) Writes your component definition to a file. 

**4.packages_to_install**: (Optional.) A list of versioned Python packages to install before running your function.

Another thing we need to consider is passing parameters between components. We can pass simple parameters such as integer, string, tuple, dict, and list by values. To pass the large datasets or complex configurations, we can use files. We can annotate the Python function’s parameters to indicate input or output files for the component. 

Refer to  https://www.kubeflow.org/docs/components/pipelines/sdk/python-function-components/ for more information.

#### Pipeline Component : Data Ingestion

In [6]:
@component(
    packages_to_install=["pandas","google-cloud-storage"],
    base_image="python:3.10.7-slim",
    output_component_file="data_ingestion.yaml"
)
def download_data(project_id: str, bucket: str, file_name: str, dataset: Output[Dataset]):
    '''download data'''
    from google.cloud import storage
    import pandas as pd
    import logging 
    import sys
    
    logging.basicConfig(stream=sys.stdout, level=logging.INFO)
    
    # Downloaing the file from a google bucket 
    client = storage.Client(project=project_id)
    bucket = client.get_bucket(bucket)
    blob = bucket.blob(file_name)
    blob.download_to_filename(dataset.path + ".csv")
    logging.info('Downloaded Data!')    
    logging.info(dataset)

#### Pipeline Component : Training LogisticRegression

In [7]:
@component(
    packages_to_install=['google-cloud-storage', 'pandas', 'scikit-learn'],
    base_image="python:3.10.7-slim",
    output_component_file="train_lr_model.yaml"
)
def train_lr (features: Input[Dataset], project_id: str, model_repo: str) -> dict:
    '''train a LogisticRegression with default parameters'''
    import pandas as pd
    from sklearn.linear_model import LogisticRegression
    from sklearn import metrics
    from sklearn.model_selection import train_test_split
    from google.cloud import storage
    import json
    import logging 
    import sys
    import os
    import pickle  
        
    logging.basicConfig(stream=sys.stdout, level=logging.INFO)
    
    logging.info('print before loading df')
    df = pd.read_csv(features.path+".csv")
    
    logging.info('df loaded')
    logging.info(df.columns)        
    
    x_train, x_test, y_train, y_test = train_test_split(df.drop('class',axis=1), 
                                                    df['class'], test_size=0.30, 
                                                    random_state=101)
    model = LogisticRegression()
    model.fit(x_train,y_train)

    metrics = {"accuracy": model.score(x_test, y_test)}
    logging.info(metrics)
   
    # Save the model localy
    local_file = '/tmp/local_model.pkl'
    with open(local_file, 'wb') as file:  
        pickle.dump(model, file)
    # write out output
  
    # Save to GCS as model.h5
    client = storage.Client(project=project_id)
    bucket = client.get_bucket(model_repo)
    blob = bucket.blob('lrmodel.pkl')
   # Upload the locally saved model
    blob.upload_from_filename(local_file)
  
    print("Saved the model to GCP bucket : " + model_repo)
    return metrics

#### Pipeline Component : Training SVC

In [8]:
@component(
    packages_to_install=['google-cloud-storage', 'pandas', 'scikit-learn'],
    base_image="python:3.10.7-slim",
    output_component_file="train_svc_model.yaml"
)
def train_svc (features: Input[Dataset], project_id: str, model_repo: str) -> dict:
    '''train a SVM with default parameters'''
    import pandas as pd
    from sklearn.svm import SVC
    from sklearn import metrics
    from sklearn.model_selection import train_test_split
    from google.cloud import storage
    import json
    import logging 
    import sys
    import os
    import pickle  
        
    logging.basicConfig(stream=sys.stdout, level=logging.INFO)
    logging.info('print before loading df')
    
    df = pd.read_csv(features.path+".csv") 
    
    logging.info('df loaded')
    
    logging.info(df.columns)        
    
    x_train, x_test, y_train, y_test = train_test_split(df.drop('class',axis=1), 
                                                    df['class'], test_size=0.30, 
                                                    random_state=101)
    logging.info('split')
    
    model = SVC()
    model.fit(x_train,y_train)

    logging.info('model fit')
    
    metrics = {"accuracy": model.score(x_test, y_test)}
    logging.info(metrics)
   
    # Save the model localy
    local_file = '/tmp/local_model.pkl'
    with open(local_file, 'wb') as file:  
        pickle.dump(model, file)
    # write out output
  
    # Save to GCS as model.h5
    client = storage.Client(project=project_id)
    bucket = client.get_bucket(model_repo)
    blob = bucket.blob('svcmodel.pkl')
   # Upload the locally saved model
    blob.upload_from_filename(local_file)
  
    print("Saved the model to GCP bucket : " + model_repo)
    return metrics

#### Pipeline Component : Prediction-LR

In [9]:
@component(
    packages_to_install=['pandas', 'scikit-learn'],
    base_image="python:3.10.7-slim",
    output_component_file="prediction_lr_3.yaml"
)
def predict_lr(project_id: str, model_repo: str, features: Input[Dataset]) -> dict:
    import pandas as pd
    import pickle  
    from google.cloud import storage
    import json
    import logging
    import sys
    import os
    
    logging.basicConfig(stream=sys.stdout, level=logging.INFO)
    
    df = pd.read_csv(features.path+".csv")
    
    client = storage.Client(project=project_id)
    bucket = client.get_bucket(model_repo)
    blob = bucket.blob('lrmodel.pkl')
    filename = '/tmp/local_model.pkl'
    blob.download_to_filename(filename)
    
    #Loading the model
    model = pickle.load(open(filename, 'rb'))

    xNew = df[['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar', 'chlorides', 'free sulfur dioxide',
 'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol']]
    
    dfcp = df.copy()   
    y_classes = model.predict(xNew)
    logging.info(type(y_classes))
    logging.info(y_classes)
    dfcp['pclass'] = y_classes.tolist()
    dic = dfcp.to_dict() 
    logging.info(len(dic))
    return dic

#### Pipeline Component : Prediction SVC

In [10]:
@component(
    packages_to_install=['pandas', 'scikit-learn'],
    base_image="python:3.10.7-slim",
    output_component_file="prediction_svc_3.yaml"
)
def predict_svc(project_id: str, model_repo: str, features: Input[Dataset]) -> dict:
    import pandas as pd
    import pickle  
    from google.cloud import storage
    import json
    import logging
    import sys
    import os
    
    logging.basicConfig(stream=sys.stdout, level=logging.INFO)
    
    df = pd.read_csv(features.path+".csv")  
    
    client = storage.Client(project=project_id)
    bucket = client.get_bucket(model_repo)
    blob = bucket.blob('svcmodel.pkl')
    filename = '/tmp/local_model.pkl'
    blob.download_to_filename(filename)
    
    #Loading the model
    model = pickle.load(open(filename, 'rb'))

    xNew = df[['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar', 'chlorides', 'free sulfur dioxide',
 'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol']]

    dfcp = df.copy()   
    y_classes = model.predict(xNew)
    logging.info(y_classes)
    dfcp['pclass'] = y_classes.tolist()
    dic = dfcp.to_dict() 
    return dic

#### Pipeline Component : Algorithm Selection 

In [11]:
@component(
    base_image="python:3.10.7-slim",
    output_component_file="compare_model.yaml"
)
def compare_model(svc_metrics: dict, lr_metrics: dict) -> str:
    import logging
    import json
    import sys
    logging.basicConfig(stream=sys.stdout, level=logging.INFO)
    logging.info(svc_metrics)
    logging.info(lr_metrics)
    if svc_metrics.get("accuracy") > lr_metrics.get("accuracy"):
        return "SVC"
    else :
        return "LR"

#### Define the Pipeline

In [12]:
# Define the workflow of the pipeline.
@kfp.dsl.pipeline(
    name="whitewine-predictor-training-pipeline",
    pipeline_root=pipeline_root_path)
def pipeline(project_id: str, data_bucket: str, trainset_filename: str, model_repo: str, testset_filename: str, ):
    
    
    di_op = download_data(
        project_id=project_id,
        bucket=data_bucket,
        file_name=trainset_filename
    )

 
    training_svc_job_run_op = train_svc(
        project_id=project_id,
        model_repo=model_repo,       
        features=di_op.outputs["dataset"]
    )
    
     
    training_lr_job_run_op = train_lr(
        project_id=project_id,
        model_repo=model_repo,       
        features=di_op.outputs["dataset"]
    )
    
    pre_di_op = download_data(
        project_id=project_id,
        bucket=data_bucket,
        file_name=testset_filename
    ).after(training_svc_job_run_op, training_lr_job_run_op)
        
        
    comp_model__op = compare_model(training_svc_job_run_op.output,
                                       training_lr_job_run_op.output).after(training_svc_job_run_op, training_lr_job_run_op)  
    
    # defining the branching condition
    with dsl.Condition(comp_model__op.output=="SVC"):
        predict_svc_job_run_op = predict_svc(
        project_id=project_id,
        model_repo=model_repo,       
        features=pre_di_op.outputs["dataset"]
        )
    with dsl.Condition(comp_model__op.output=="LR"):
        predict_lr_job_run_op = predict_lr(
        project_id=project_id,
        model_repo=model_repo,       
        features=pre_di_op.outputs["dataset"]
       )

#### Compile the pipeline into a JSON file

In [13]:
from kfp.v2 import compiler
compiler.Compiler().compile(pipeline_func=pipeline,
        package_path='whitewine_predictor_training_pipeline.json')

/home/jupyter/.local/lib/python3.7/site-packages/kfp/v2/compiler/compiler.py:1293: FutureWarning: APIs imported from the v1 namespace (e.g. kfp.dsl, kfp.components, etc) will not be supported by the v2 compiler since v2.0.0
  category=FutureWarning,


#### Submit the pipeline run

In [14]:
import google.cloud.aiplatform as aip

job = aip.PipelineJob(
    display_name="whitewine-predictor",
    enable_caching=False,
    template_path="whitewine_predictor_training_pipeline.json",
    pipeline_root=pipeline_root_path,
    parameter_values={
        'project_id': project_id, # makesure to use your project id 
        'data_bucket': 'data_de2022_2093373',  # makesure to use your data bucket name 
        'trainset_filename': 'training_set.csv',     # makesure to upload these to your data bucket from DE2022/lab4/data
        'testset_filename': 'prediction_set.csv',    # makesure to upload these to your data bucket from DE2022/lab4/data
        'model_repo':'model_repo_de2022_2093373' # makesure to use your model bucket name 
    }
)

job.run()

Creating PipelineJob
PipelineJob created. Resource name: projects/171238576493/locations/us-central1/pipelineJobs/whitewine-predictor-training-pipeline-20221022141904
To use this PipelineJob in another session:
pipeline_job = aiplatform.PipelineJob.get('projects/171238576493/locations/us-central1/pipelineJobs/whitewine-predictor-training-pipeline-20221022141904')
View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/whitewine-predictor-training-pipeline-20221022141904?project=171238576493
PipelineJob projects/171238576493/locations/us-central1/pipelineJobs/whitewine-predictor-training-pipeline-20221022141904 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projects/171238576493/locations/us-central1/pipelineJobs/whitewine-predictor-training-pipeline-20221022141904 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projects/171238576493/locations/us-central1/pipelineJobs/whitewine-predictor-training-pipeline-202210

RuntimeError: Job failed with:
code: 9
message: "The DAG failed because some tasks failed. The failed tasks are: [condition-2].; Job (project_id = de2022-362707, job_id = 8795328861626695680) is failed due to the above error.; Failed to handle the job: {project_number = 171238576493, job_id = 8795328861626695680}"
